# Task 4.5 Data Consistency Checks

In [1]:
import pandas as pd
import numpy as np
import os

In [41]:
df_ords = pd.read_csv(r'C:\Users\RUZANNA\OneDrive\Desktop\Careerfoundry\Python\Instacart Basket Anlysis-05.2023\02_Data\Prepared Data\orders_wrangled.csv', index_col = False)

In [53]:
path = r'C:\Users\RUZANNA\OneDrive\Desktop\Careerfoundry\Python\Instacart Basket Anlysis-05.2023\02_Data'

## Step 2

In [42]:
df_ords.describe()

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.710542e+06,1.029782e+05,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,9.875817e+05,5.953372e+04,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,8.552715e+05,5.139400e+04,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.710542e+06,1.026890e+05,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.565812e+06,1.543850e+05,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,3.421083e+06,2.062090e+05,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


No  idea how this "Unnamed:0" column is appeared. I tried to deleate it with df.drop(columns = ['Unnamed:0']) function but it returs error. 

In [43]:
df_ords['user_id'] = df_ords['user_id'].astype('str')
df_ords['order_id']= df_ords['order_id'].astype('str')

In [44]:
df_ords.describe()

,Unnamed: 0,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
count,3.421083e+06,3.421083e+06,3.421083e+06,3.421083e+06,3.214874e+06
mean,1.710541e+06,1.715486e+01,2.776219e+00,1.345202e+01,1.111484e+01
std,9.875817e+05,1.773316e+01,2.046829e+00,4.226088e+00,9.206737e+00
min,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,8.552705e+05,5.000000e+00,1.000000e+00,1.000000e+01,4.000000e+00
50%,1.710541e+06,1.100000e+01,3.000000e+00,1.300000e+01,7.000000e+00
75%,2.565812e+06,2.300000e+01,5.000000e+00,1.600000e+01,1.500000e+01
max,3.421082e+06,1.000000e+02,6.000000e+00,2.300000e+01,3.000000e+01


In [13]:
# I think "Order_Number" column looks strange, where min and max number of orders are the same: '1', whereas 25% is '5'. 
# That column needs to be investigated further

# Step 3

In [45]:
for col in df_ords.columns.tolist():
  weird = (df_ords[[col]].applymap(type) != df_ords[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (df_ords[weird]) > 0:
    print (col)

# Step 4

No mixed-type data found 

# Step 5

In [46]:
# identifying missing data
df_ords.isnull().sum()

Unnamed: 0                     0
order_id                       0
user_id                        0
order_number                   0
orders_day_of_week             0
order_hour_of_day              0
days_since_prior_order    206209
dtype: int64

In [47]:
df_ords.shape

(3421083, 7)

In the column 'days_since_prior_order' we found 206209 missing values. We cannot replace the missing values with mean or median, as it is about exact day since prior order of customer. We need extra information , if 'NaN' means zero we can replace it.


In [48]:
df_ords.head(10)

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,0,2539329,1,1,2,8,NaN
1,1,2398795,1,2,3,7,15.0
2,2,473747,1,3,3,12,21.0
3,3,2254736,1,4,4,7,29.0
4,4,431534,1,5,4,15,28.0
5,5,3367565,1,6,2,7,19.0
6,6,550135,1,7,1,9,20.0
7,7,3108588,1,8,1,14,14.0
8,8,2295261,1,9,1,16,0.0
9,9,2550362,1,10,4,8,30.0


# Step 6

In [51]:
df_ords_clean = df_ords[df_ords['days_since_prior_order'].isnull()== False]
df_ords_clean.shape

(3214874, 7)

As the missing values in 'days_since_prior_order' column is less than 10 % of data (raws number) we can just ignore it and create a new dataframe without missing values.

# Step 7

In [50]:
df_duplicate = df_ords_clean[df_ords_clean.duplicated()]
df_duplicate

,Unnamed: 0,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order


 No duplicates were found.

# Step 8

No duplicates were found, but if there were any duplicated raws we would delete it with the use of a function df.drop_duplicates()

# Step 9

In [54]:
df_ords_clean.to_csv(os.path.join(path, 'Prepared Data', 'orders_checked.csv'))